In [12]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd

In [3]:
train_ds_feat = pd.read_csv('../../data/processed_train_feat.csv')
train_ds_targ = pd.read_csv('../../data/processed_train_targ.csv')

test_ds_feat = pd.read_csv('../../data/processed_test_feat.csv')
test_ds_targ = pd.read_csv('../../data/processed_test_targ.csv')

In [16]:
len(train_ds_targ.iloc[:, 1:].values) == len(train_ds_feat.iloc[:, 1:].values)
print(len(train_ds_feat.columns))

40


In [10]:
class MyDataset(Dataset):
    def __init__(self, feat, targ):
        self.feat = feat.iloc[:, 1:].values
        self.targ = targ.iloc[:, 1:].values
        
    def __len__(self):
        return len(self.feat)
    
    def __getitem__(self, idx):
        feat = torch.FloatTensor(self.feat[idx])
        targ = torch.FloatTensor(self.targ[idx])
        return feat, targ

In [17]:
train_ds = MyDataset(train_ds_feat, train_ds_targ)
test_ds = MyDataset(test_ds_feat, test_ds_targ)

train_dl = DataLoader(train_ds, batch_size=1, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=1, shuffle=True)

In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [19]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, output_size)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork(len(train_ds_feat.columns)-1, len(train_ds_targ.columns)-1).to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=39, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=2, bias=True)
  )
)


In [20]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [21]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [22]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [23]:
epochs = 20

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dl, model, loss_fn, optimizer)
    test(test_dl, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 120.560211  [    0/ 1485]
loss: 45.529320  [  100/ 1485]
loss: 49.926876  [  200/ 1485]
loss: 217.942169  [  300/ 1485]
loss: 115.530121  [  400/ 1485]
loss: 42.695717  [  500/ 1485]
loss: 235.472092  [  600/ 1485]
loss: 174.595245  [  700/ 1485]
loss: 104.867661  [  800/ 1485]
loss: 52.660538  [  900/ 1485]
loss: 233.681488  [ 1000/ 1485]
loss: 289.930695  [ 1100/ 1485]
loss: 35.166996  [ 1200/ 1485]
loss: 112.259674  [ 1300/ 1485]
loss: 172.570663  [ 1400/ 1485]
Test Error: 
 Accuracy: 0.0%, Avg loss: 161.628093 

Epoch 2
-------------------------------
loss: 72.218475  [    0/ 1485]
loss: 49.862251  [  100/ 1485]
loss: 23.159868  [  200/ 1485]
loss: 210.594574  [  300/ 1485]
loss: 271.214752  [  400/ 1485]
loss: 145.569794  [  500/ 1485]
loss: 139.796448  [  600/ 1485]
loss: 71.326935  [  700/ 1485]
loss: 209.242584  [  800/ 1485]
loss: 135.485962  [  900/ 1485]
loss: 148.723541  [ 1000/ 1485]
loss: 81.837822  [ 1100/ 1485]
loss: 167.661